In [ ]:
import os
from openai import OpenAI
import rich
import requests
from io import BytesIO
from google.colab import userdata

In [ ]:
api_key = userdata.get('OPENAI_API_KEY')
MODEL = "gpt-4o-mini"

openai = OpenAI(api_key=api_key)

## File Search is only part of Responses API

https://platform.openai.com/docs/guides/tools-file-search

File Search works with a vector store, which we will need to create within OpenAI. The entire process, including storing the file, generating embeddings, and searching within the vector store, will be managed by OpenAI



---



The `create_file` function uploads a file to the OpenAI API and supports both remote and local files.
If the file path is a URL, it downloads the file content, stores it in memory and then upload

In [ ]:
# Create function to Upload a file
def create_file(openai, file_path):
    if file_path.startswith("http://") or file_path.startswith("https://"):
        # Download the file content from the URL
        response = requests.get(file_path)
        file_content = BytesIO(response.content)
        file_name = file_path.split("/")[-1]
        file_tuple = (file_name, file_content)
        result = openai.files.create(
            file=file_tuple,
            purpose="assistants"
        )
    else:
        # Handle local file path
        with open(file_path, "rb") as file_content:
            result = openai.files.create(
                file=file_content,
                purpose="assistants"
            )
    print(result.id)
    return result.id

##### Call function to upload a file

In [ ]:
file_id = create_file(openai, "https://cdn.openai.com/API/docs/deep_research_blog.pdf")
# file_id = create_file(openai, "docs/Panaversity-Certified-Agentic-and-Robotic-AI-Engineer.pdf")

file-BE3vg12ZPEvwT2R6EGEoCu


### Create a vector store

In [ ]:
vector_store = openai.vector_stores.create(
    name="knowledge_base_2"
)
print(vector_store.id)

vs_67e3905592188191bc32abea607597bf


### Add a file to a vector store

In [ ]:
result = openai.vector_stores.files.create(
    vector_store_id=vector_store.id,
    file_id=file_id
)
rich.print(result)

VectorStoreFile(
    id='file-BE3vg12ZPEvwT2R6EGEoCu',
    created_at=1742966891,
    last_error=None,
    object='vector_store.file',
    status='in_progress',
    usage_bytes=0,
    vector_store_id='vs_67e3905592188191bc32abea607597bf',
    attributes={},
    chunking_strategy=StaticFileChunkingStrategyObject(
        static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800),
        type='static'
    )
)

### Check the status of files in vector store

In [ ]:
result = openai.vector_stores.files.list(
    vector_store_id=vector_store.id
)
rich.print(result)

SyncCursorPage[VectorStoreFile](
    data=[
        VectorStoreFile(
            id='file-BE3vg12ZPEvwT2R6EGEoCu',
            created_at=1742966891,
            last_error=None,
            object='vector_store.file',
            status='completed',
            usage_bytes=66539,
            vector_store_id='vs_67e3905592188191bc32abea607597bf',
            attributes={},
            chunking_strategy=StaticFileChunkingStrategyObject(
                static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800),
                type='static'
            )
        )
    ],
    has_more=False,
    object='list',
    first_id='file-BE3vg12ZPEvwT2R6EGEoCu',
    last_id='file-BE3vg12ZPEvwT2R6EGEoCu'
)

Now in Responses API call, we can provide `file_search` tool and `vector_store_ids`.

Annotation object in response will provide information about file
```
AnnotationFileCitation(
    file_id='file-4GoREtVZ14bPdCKHXsva8k',
    index=1939,
    type='file_citation',
    filename='deep_research_blog.pdf'
)
```

In [ ]:
response = openai.responses.create(
    model=MODEL,
    input="What is deep research by OpenAI?",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store.id]
    }]
)
rich.print(response.output[1].content[0].text)
# rich.print(response)
rich.print(response.output)

Deep Research by OpenAI is a new capability designed to assist users in conducting complex, multi-step research 
tasks on the internet. Here are the key features and functionalities:

1. **Synthesis of Information**: Deep Research allows users to prompt ChatGPT, which then independently finds, 
analyzes, and synthesizes information from numerous online sources to generate comprehensive reports, akin to the 
work of a research analyst.

2. **Time Efficiency**: It is capable of accomplishing tasks that could take a human hours in just tens of minutes,
thereby saving valuable time for users engaged in intensive knowledge work across various domains like finance, 
science, and policy.

3. **Technical Capabilities**: The capability is powered by a version of OpenAI's o3 model, optimized for web 
browsing and data analysis. It uses advanced reasoning to search, interpret, and analyze a vast array of data, 
including text, images, and PDFs.

4. **Output Documentation**: Outputs are fully documented, featuring clear citations and a summary of the findings,
which enhances reliability and verifiability.

5. **User Accessibility**: Currently, Deep Research is available to Pro users, with planned rollouts to Plus and 
Team users. 

6. **Recognized Evaluation Performance**: In evaluations like "Humanity's Last Exam," the model behind Deep 
Research achieved a notable accuracy of 26.6%, marking a significant advancement in AI's ability to handle 
expert-level tasks.

This initiative signifies OpenAI's commitment to developing more advanced tools that enhance productivity and 
knowledge generation, moving closer to achieving Artificial General Intelligence (AGI).

[
    ResponseFileSearchToolCall(
        id='fs_67e3909db5588192abe02a30d9c7ee4902b275aee71d1c0e',
        queries=[
            'What is deep research by OpenAI?',
            'OpenAI deep research overview',
            'OpenAI research initiatives'
        ],
        status='completed',
        type='file_search_call',
        results=None
    ),
    ResponseOutputMessage(
        id='msg_67e390a18828819283f95e795731a0e802b275aee71d1c0e',
        content=[
            ResponseOutputText(
                annotations=[
                    AnnotationFileCitation(
                        file_id='file-BE3vg12ZPEvwT2R6EGEoCu',
                        index=434,
                        type='file_citation',
                        filename='deep_research_blog.pdf'
                    ),
                    AnnotationFileCitation(
                        file_id='file-BE3vg12ZPEvwT2R6EGEoCu',
                        index=684,
                        type='file_citation',
                        filename='deep_research_blog.pdf'
                    ),
                    AnnotationFileCitation(
                        file_id='file-BE3vg12ZPEvwT2R6EGEoCu',
                        index=684,
                        type='file_citation',
                        filename='deep_research_blog.pdf'
                    ),
                    AnnotationFileCitation(
                        file_id='file-BE3vg12ZPEvwT2R6EGEoCu',
                        index=940,
                        type='file_citation',
                        filename='deep_research_blog.pdf'
                    ),
                    AnnotationFileCitation(
                        file_id='file-BE3vg12ZPEvwT2R6EGEoCu',
                        index=940,
                        type='file_citation',
                        filename='deep_research_blog.pdf'
                    ),
                    AnnotationFileCitation(
                        file_id='file-BE3vg12ZPEvwT2R6EGEoCu',
                        index=1103,
                        type='file_citation',
                        filename='deep_research_blog.pdf'
                    ),
                    AnnotationFileCitation(
                        file_id='file-BE3vg12ZPEvwT2R6EGEoCu',
                        index=1103,
                        type='file_citation',
                        filename='deep_research_blog.pdf'
                    ),
                    AnnotationFileCitation(
                        file_id='file-BE3vg12ZPEvwT2R6EGEoCu',
                        index=1229,
                        type='file_citation',
                        filename='deep_research_blog.pdf'
                    ),
                    AnnotationFileCitation(
                        file_id='file-BE3vg12ZPEvwT2R6EGEoCu',
                        index=1229,
                        type='file_citation',
                        filename='deep_research_blog.pdf'
                    ),
                    AnnotationFileCitation(
                        file_id='file-BE3vg12ZPEvwT2R6EGEoCu',
                        index=1466,
                        type='file_citation',
                        filename='deep_research_blog.pdf'
                    ),
                    AnnotationFileCitation(
                        file_id='file-BE3vg12ZPEvwT2R6EGEoCu',
                        index=1466,
                        type='file_citation',
                        filename='deep_research_blog.pdf'
                    ),
                    AnnotationFileCitation(
                        file_id='file-BE3vg12ZPEvwT2R6EGEoCu',
                        index=1665,
                        type='file_citation',
                        filename='deep_research_blog.pdf'
                    ),
                    AnnotationFileCitation(
                        file_id='file-BE3vg12ZPEvwT2R6EGEoCu',
                        index=1665,
                        type='file_citation',
                        filename='deep_research_b

Customize the number of results you want to retrieve from the vector stores.


You will notice a change in the result only if you also provide `include=["file_search_call.results"]`. This will allow you to see the difference.

In [ ]:
response = openai.responses.create(
    model=MODEL,
    input="What is deep research by OpenAI?",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store.id],
        "max_num_results": 2
    }],
    include=["file_search_call.results"]  # This will include actual portion of text content from the file
)
rich.print(response.output[1].content[0].text)
# rich.print(response)
rich.print(response.output[0].results)
rich.print(response.output[1])

Deep Research by OpenAI is a new capability integrated into ChatGPT, designed to handle complex, multi-step 
research tasks efficiently. It allows users to conduct thorough web research by synthesizing information from 
numerous online sources quickly, achieving results in minutes that would typically take hours for a human.

Here are some key features of Deep Research:

1. **Autonomous Research Agent**: It independently finds, analyzes, and synthesizes vast amounts of data to create 
comprehensive reports comparable to a research analyst's output.
   
2. **Reasoning and Synthesis**: Deep Research is capable of reasoning to examine large datasets, interpret various 
forms of content (text, images, PDFs), and pivot its approach based on new findings.

3. **Broad Application**: It's particularly beneficial for professionals in finance, science, policy, and 
engineering, but can also assist consumers searching for detailed recommendations and analyses.

4. **Documentation and Citation**: Every output comes with clear citations and a summary of the reasoning process, 
enhancing transparency and reliability.

5. **Tools and Training**: It was trained using real-world tasks that require web browsing and data analysis, 
employing advanced reinforcement learning methods. This allows it to efficiently execute multi-step 
information-gathering tasks. 

Overall, Deep Research represents a significant advancement towards OpenAI’s goal of developing artificial general 
intelligence (AGI) by enhancing the ability to synthesize knowledge and produce novel outputs.

[
    Result(
        attributes={},
        file_id='file-BE3vg12ZPEvwT2R6EGEoCu',
        filename='deep_research_blog.pdf',
        score=0.9820093016280954,
        text="Introducing deep research | OpenAI\n\n\nFebruary 2, 2025 Release\n\nIntroducing deep research\nAn 
agent that uses reasoning to synthesize large amounts of\nonline information and complete multi-step research 
tasks\nfor you. Available to Pro users today, Plus and Team next.\n\nTry on ChatGPT\n\nListen to article 8:18 
Share\n\n21/02/2025, 19:58 Introducing deep research | 
OpenAI\n\nhttps://openai.com/index/introducing-deep-research/ 
1/38\n\nhttps://openai.com/research/index/release/\nhttps://chatgpt.com/\nhttps://openai.com/\n\n\nToday we’re 
launching deep research in ChatGPT, a new agentic capability that\nconducts multi-step research on the internet for
complex tasks. It accomplishes in\ntens of minutes what would take a human many hours.\n\nDeep research is OpenAI's
next agent that can do work for you independently—you\ngive it a prompt, and ChatGPT will find, analyze, and 
synthesize hundreds of online\nsources to create a comprehensive report at the level of a research analyst. 
Powered\nby a version of the upcoming OpenAI o3 model that’s optimized for web browsing and\ndata analysis, it 
leverages reasoning to search, interpret, and analyze massive amounts\nof text, images, and PDFs on the internet, 
pivoting as needed in reaction to information\nit encounters.\n\nThe ability to synthesize knowledge is a 
prerequisite for creating new knowledge. For\nthis reason, deep research marks a significant step toward our 
broader goal of\ndeveloping AGI, which we have long envisioned as capable of producing novel\nscientific 
research.\n\nWhy we built deep research\n\nDeep research is built for people who do intensive knowledge work in 
areas like\nfinance, science, policy, and engineering and need thorough, precise, and reliable\nresearch. It can be
equally useful for discerning shoppers looking for hyper-\npersonalized recommendations on purchases that typically
require careful research,\nlike cars, appliances, and furniture. Every output is fully documented, with 
clear\ncitations and a summary of its thinking, making it easy to reference and verify the\ninformation. It is 
particularly effective at finding niche, non-intuitive information that\nwould require browsing numerous websites. 
Deep research frees up valuable time by\nallowing you to offload and expedite complex, time-intensive web research 
with just\none query.\n\nDeep research independently discovers, reasons about, and consolidates insights 
from\nacross the web. To accomplish this, it was trained on real-world tasks requiring\nbrowser and Python tool 
use, using the same reinforcement learning methods behind\nOpenAI o1, our first reasoning model. While o1 
demonstrates impressive capabilities in\n\n21/02/2025, 19:58 Introducing deep research | 
OpenAI\n\nhttps://openai.com/index/introducing-deep-research/ 2/38\n\nhttps://openai.com/\n\n\ncoding, math, and 
other technical domains, many real-world challenges demand\nextensive context and information gathering from 
diverse online sources. Deep\nresearch builds on these reasoning capabilities to bridge that gap, allowing it to 
take\non the types of problems people face in work and everyday life.\n\nHow to use deep research\n\nIn ChatGPT, 
select ‘deep research’ in the message composer and enter your query. Tell\nChatGPT what you need—whether it’s a 
competitive analysis on streaming platforms\nor a personalized report on the best commuter bike. You can attach 
files or\nspreadsheets to add context to your question. Once it starts running, a sidebar\nappears with a summary 
of the steps taken and sources used."
    ),
    Result(
        attributes={},
        file_id='file-BE3vg12ZPEvwT2R6EGEoCu',
        filename='deep_research_blog.pdf',
        score=0.946322956540013,
        text="Deep research\n\nHelp me find iOS and android adopti

ResponseOutputMessage(
    id='msg_67e390bdf97481928c6b0d829a885c110dcae4f636bbcc19',
    content=[
        ResponseOutputText(
            annotations=[
                AnnotationFileCitation(
                    file_id='file-BE3vg12ZPEvwT2R6EGEoCu',
                    index=1353,
                    type='file_citation',
                    filename='deep_research_blog.pdf'
                ),
                AnnotationFileCitation(
                    file_id='file-BE3vg12ZPEvwT2R6EGEoCu',
                    index=1565,
                    type='file_citation',
                    filename='deep_research_blog.pdf'
                )
            ],
            text="Deep Research by OpenAI is a new capability integrated into ChatGPT, designed to handle complex, 
multi-step research tasks efficiently. It allows users to conduct thorough web research by synthesizing information
from numerous online sources quickly, achieving results in minutes that would typically take hours for a 
human.\n\nHere are some key features of Deep Research:\n\n1. **Autonomous Research Agent**: It independently finds,
analyzes, and synthesizes vast amounts of data to create comprehensive reports comparable to a research analyst's 
output.\n   \n2. **Reasoning and Synthesis**: Deep Research is capable of reasoning to examine large datasets, 
interpret various forms of content (text, images, PDFs), and pivot its approach based on new findings.\n\n3. 
**Broad Application**: It's particularly beneficial for professionals in finance, science, policy, and engineering,
but can also assist consumers searching for detailed recommendations and analyses.\n\n4. **Documentation and 
Citation**: Every output comes with clear citations and a summary of the reasoning process, enhancing transparency 
and reliability.\n\n5. **Tools and Training**: It was trained using real-world tasks that require web browsing and 
data analysis, employing advanced reinforcement learning methods. This allows it to efficiently execute multi-step 
information-gathering tasks. \n\nOverall, Deep Research represents a significant advancement towards OpenAI’s goal 
of developing artificial general intelligence (AGI) by enhancing the ability to synthesize knowledge and produce 
novel outputs.",
            type='output_text'
        )
    ],
    role='assistant',
    status='completed',
    type='message'
)